In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import numpy as np
from keras.utils import to_categorical
from collections import Counter
!pip install transformers torch scikit-learn
from sklearn.preprocessing import LabelEncoder
from itertools import chain
from sklearn.preprocessing import OneHotEncoder
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.utils import resample
from tensorflow.keras.preprocessing.text import one_hot
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
import unicodedata
from gensim.models.fasttext import FastText
from sklearn.metrics import roc_curve,auc, roc_auc_score, cohen_kappa_score, brier_score_loss
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dropout
from tensorflow.keras.layers import Dense, Bidirectional, LSTM
import tensorflow as tf
from itertools import cycle
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
%matplotlib inline
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.preprocessing import MultiLabelBinarizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!pip install transformers[torch] accelerate -U scikit-learn

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df=pd.read_csv("data_1C.csv")
df.head()

,Unnamed: 0,index,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
0,0,13012,Subhan?Allah Kenangan yg Tidak akan Terlupakan...,1,0,1,0,0,0,0,0,1
1,1,5594,"Direksi Pertamina Dirombak, ESDM Mau Premium t...",0,0,0,0,0,0,0,0,0
2,2,347,"3. Karena berbasis teknologi digital, Presiden...",0,0,0,0,0,0,0,0,0
3,3,4356,RT USER: USER USER USER kebanyakan alasan...\n...,1,1,1,0,0,0,0,1,0
4,4,10958,Ganyang PKI dg segala anteknya... URL,1,0,0,1,0,1,0,0,0


In [ ]:
df = df.drop(columns=['Unnamed: 0', 'index'])

In [ ]:
def clean_text(text):
    if isinstance(text, str):
        # Menghapus spasi yang double
        text = re.sub(r'\s+', ' ', text).strip()
        # Menghapus data yang mengandung angka
        text = re.sub(r'\d+', '', text)
        # Menghapus data yang mengandung tanda baca
        text = re.sub(r'[^\w\s]', '', text)
        # Mengubah semua teks di dalam data menjadi lowercase
        text = text.lower()
        return text
    else:
        return text  # or return an empty string, or handle NaNs in another way
df['Tweet'] = df['Tweet'].apply(clean_text)

# Define features and labels
X = df['Tweet'].values
y = df.drop(columns=['Tweet']).values

In [ ]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform([tuple(row) for row in y])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
class TweetDataset(Dataset):
    def __init__(self, tweets, labels, tokenizer, max_len):
        self.tweets = tweets
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, idx):
        tweet = self.tweets[idx]
        labels = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            tweet,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'tweet_text': tweet,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.float)
        }

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
MAX_LEN = 128
train_dataset = TweetDataset(X_train, y_train, tokenizer, MAX_LEN)
test_dataset = TweetDataset(X_test, y_test, tokenizer, MAX_LEN)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)

In [ ]:
def compute_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = (preds > 0.5).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='micro')
    acc = accuracy_score(p.label_ids, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [ ]:
learning_rates = [2e-5, 3e-5]
num_train_epochs_list = [5, 10]

best_f1 = 0
best_params = {}

for learning_rate in learning_rates:
    for num_train_epochs in num_train_epochs_list:
        model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2, problem_type="multi_label_classification")

        training_args = TrainingArguments(
            output_dir='./results',
            num_train_epochs=num_train_epochs,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            warmup_steps=500,
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=10,
            evaluation_strategy="epoch",
            learning_rate=learning_rate
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
            compute_metrics=compute_metrics
        )

        # Train the model
        trainer.train()

        # Evaluate the model
        eval_result = trainer.evaluate()

        if eval_result['eval_f1'] > best_f1:
            best_f1 = eval_result['eval_f1']
            best_params = {
                'learning_rate': learning_rate,
                'num_train_epochs': num_train_epochs
            }

print(f"Best hyperparameters : {best_params}")


model.safetensors:  62%|######1   | 273M/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.362300,0.346787,0.537449,0.828646,0.974427,0.720809
2,0.257400,0.291032,0.706478,0.900206,0.952658,0.853229
3,0.191400,0.247869,0.778340,0.926633,0.952479,0.902153
4,0.161900,0.263372,0.795547,0.934839,0.924697,0.945205
5,0.110700,0.298329,0.802632,0.935919,0.943046,0.928898


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.364700,0.354918,0.663968,0.891076,0.896370,0.885845
2,0.276100,0.311639,0.646761,0.875045,0.969841,0.797130
3,0.258200,0.251841,0.744939,0.914634,0.951374,0.880626
4,0.156000,0.247843,0.790486,0.932595,0.931079,0.934116
5,0.105900,0.395709,0.758097,0.926755,0.873988,0.986301
6,0.052900,0.342607,0.800607,0.934399,0.954422,0.915199
7,0.023000,0.363365,0.810729,0.938749,0.942763,0.934768
8,0.005200,0.424915,0.813765,0.940026,0.939414,0.940639
9,0.003500,0.454544,0.803644,0.936184,0.944260,0.928245
10,0.003300,0.465306,0.800607,0.935005,0.945928,0.924331


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.368600,0.343047,0.688259,0.900194,0.894398,0.906067
2,0.277300,0.273482,0.750000,0.919935,0.914304,0.925636
3,0.228000,0.287841,0.739879,0.910484,0.976831,0.852577
4,0.156600,0.259346,0.798583,0.936279,0.919497,0.953686
5,0.092200,0.268948,0.814777,0.940177,0.942333,0.938030


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.346100,0.344915,0.680162,0.901250,0.865027,0.940639
2,0.271300,0.320855,0.649798,0.876163,0.970658,0.798434
3,0.221700,0.314182,0.689271,0.889846,0.988836,0.808871
4,0.127100,0.259080,0.790486,0.930280,0.961699,0.900848
5,0.196500,0.319724,0.789474,0.935041,0.896944,0.976517
6,0.024700,0.336966,0.814777,0.939703,0.949401,0.930202
7,0.034200,0.397983,0.821862,0.941914,0.953240,0.930855
8,0.002600,0.437221,0.817814,0.939920,0.962406,0.918461
9,0.002200,0.438083,0.824899,0.943223,0.949141,0.937378
10,0.002200,0.441936,0.825911,0.943607,0.948583,0.938682


Best hyperparameters : {'learning_rate': 3e-05, 'num_train_epochs': 10}


In [ ]:
# Train the final model with the best hyperparameters
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2, problem_type="multi_label_classification")

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.05,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    learning_rate=3e-5
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the final model
trainer.train()

# Evaluate the final model
results = trainer.evaluate()

# Display the results
print(results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.342000,0.338341,0.686235,0.902454,0.871733,0.935421
2,0.249600,0.269007,0.730769,0.909153,0.954122,0.868232
3,0.197200,0.301607,0.747976,0.913572,0.976261,0.858447
4,0.163300,0.265208,0.815789,0.940755,0.938921,0.942596
5,0.073700,0.372841,0.814777,0.941176,0.927757,0.954990
6,0.051700,0.321538,0.822874,0.941918,0.958784,0.925636
7,0.032200,0.410498,0.826923,0.943359,0.958277,0.928898
8,0.009400,0.419508,0.820850,0.941565,0.953209,0.930202
9,0.002000,0.457882,0.826923,0.943321,0.958895,0.928245
10,0.001800,0.449302,0.826923,0.943620,0.954000,0.933464


{'eval_loss': 0.4493021070957184, 'eval_accuracy': 0.8269230769230769, 'eval_f1': 0.9436201780415431, 'eval_precision': 0.954, 'eval_recall': 0.9334637964774951, 'eval_runtime': 7.1567, 'eval_samples_per_second': 138.053, 'eval_steps_per_second': 8.663, 'epoch': 10.0}


In [ ]:
print("Evaluation results:")
print(results)

# Additional evaluation on the test set with detailed metrics
predictions = trainer.predict(test_dataset)
preds = (predictions.predictions > 0.5).astype(int)

# Calculate detailed metrics
precision, recall, f1, _ = precision_recall_fscore_support(y_test, preds, average='micro')
accuracy = accuracy_score(y_test, preds)

print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
print(f"Test F1 Score: {f1}")

Evaluation results:
{'eval_loss': 0.4493021070957184, 'eval_accuracy': 0.8269230769230769, 'eval_f1': 0.9436201780415431, 'eval_precision': 0.954, 'eval_recall': 0.9334637964774951, 'eval_runtime': 7.1567, 'eval_samples_per_second': 138.053, 'eval_steps_per_second': 8.663, 'epoch': 10.0}


Test Accuracy: 0.8269230769230769
Test Precision: 0.954
Test Recall: 0.9334637964774951
Test F1 Score: 0.9436201780415431
